In [1]:
#!pip install pytse_client

In [30]:
import pytse_client as tse
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [31]:
#Download data from TSE Site
data = tse.download(symbols=["کچاد"],include_jdate= True,adjust = False) #adjust = True :قیمت تعدیل یافته سهم

In [84]:
df=data['کچاد']

In [85]:
a= np.empty(([df.shape[0],1]),dty)


In [86]:
df = df +a

UFuncTypeError: ufunc 'add' cannot use operands with types dtype('<M8[ns]') and dtype('float64')

In [41]:
df

,date,open,high,low,adjClose,value,volume,count,yesterday,close,jdate,3916,3917,3921
0,2003-10-19,2991.0,2991.0,2900.0,2916.0,5833615016,2000000,667,0.0,2916.0,1382-07-27,None,None,None
1,2003-10-20,3061.0,3061.0,3061.0,3061.0,22957500000,7500000,1277,2916.0,3061.0,1382-07-28,None,None,None
2,2003-10-21,3214.0,3214.0,3214.0,3214.0,19284000000,6000000,1102,3061.0,3214.0,1382-07-29,None,None,None
3,2003-10-22,3374.0,3374.0,3374.0,3374.0,10122000000,3000000,670,3214.0,3374.0,1382-07-30,None,None,None
4,2003-10-25,3542.0,3542.0,3542.0,3542.0,33039067600,9327800,1540,3374.0,3542.0,1382-08-03,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,2022-02-07,29150.0,29870.0,29010.0,29200.0,49884445020,1705522,853,29160.0,29200.0,1400-11-18,None,None,None
3912,2022-02-08,29690.0,29890.0,29030.0,29580.0,112509287780,3800062,694,29200.0,29570.0,1400-11-19,None,None,None
3913,2022-02-09,30000.0,30000.0,29410.0,29620.0,52042600390,1754238,891,29580.0,29660.0,1400-11-20,None,None,None
3914,2022-02-12,29220.0,30000.0,29220.0,29660.0,41688325070,1401533,704,29620.0,29650.0,1400-11-23,None,None,None


In [ ]:
type(df),type(data)

In [ ]:
df.describe()

In [ ]:
#Tenkan-sen = (max-9-period high + min-9-period low) / 2
tenkan_max = df['high'].rolling(window = 9, min_periods = 0).max()
tenkan_min = df['low'].rolling(window = 9, min_periods = 0).min()
df['tenkan_sen'] = (tenkan_max + tenkan_min) / 2


In [ ]:
#Kijun-sen = (max-26-period high + min-26-period low) / 2
kijun_max = df['high'].rolling(window = 26, min_periods = 0).max()
kijun_min = df['low'].rolling(window = 26, min_periods = 0).min()
df['kijun_sen'] = (kijun_max + kijun_min) / 2


In [ ]:
#Senkou Span A = (Tenkan-sen + Kijun-sen) / 2
#(Kijun + Tenkan) / 2 Shifted ahead by 26 periods
df['senkou_a'] = ((df['kijun_sen'] + df['tenkan_sen']) / 2).shift(26)


In [ ]:
#Senkou Span B = (max-52-period high + max-52-period low) / 2
#52 period High + Low / 2
senkou_b_max = df['high'].rolling(window = 52, min_periods = 0).max()
senkou_b_min = df['low'].rolling(window = 52, min_periods = 0).min()
df['senkou_b'] = ((senkou_b_max + senkou_b_min) / 2).shift(52)

In [ ]:
#Chikou Span = Close plotted 26 periods in the past
#Current close shifted -26
df['chikou'] = (df['close']).shift(-26)


In [ ]:
df.head()

In [ ]:
#Plotting Ichimoku
tmp = df[['close','kijun_sen','tenkan_sen','senkou_a','senkou_b','chikou']].tail(200)
fig,ax = plt.subplots(1,1,sharex=True,figsize = (20,9)) #share x axis and set a figure size
ax.plot(tmp,label=tmp.columns,linewidth=2) # plot Close with index on x-axis with a line thickness of 4
ax.fill_between(tmp.index, tmp.senkou_a, tmp.senkou_b)
ax.legend()

In [ ]:
tmp